In [27]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.diagnostic import linear_rainbow, linear_harvey_collier, het_breuschpagan
from statsmodels.stats.stattools import jarque_bera
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

Z formalnego, statystycznego punktu widzenia regresja liniowa czyni szereg założeń ([Wikipedia](https://en.wikipedia.org/wiki/Linear_regression#Assumptions)):
1. Liniowość - relacja w danych może być reprezentowana jako `y=Xw`.
2. Normalność błędów - błędy (rezydua) mają rozkład normalny, wycentrowany na zerze.
3. Homoskedastyczność (stała wariancja) - wariancja błędu nie zależy od wartości docelowych `y`. Innymi słowy, nasz błąd będzie w przybliżeniu miał podobny "rozrzut" dla małych i dużych wartości `y`.
4. Niezależność błędów - błąd i `y` są niezależne (w sensie statystycznym). Innymi słowy, nie ma między nimi bezpośredniej relacji. Jeżeli nie pracujemy z szeregami czasowymi, to to założenie po prostu jest spełnione.
5. Brak współliniowości zmiennych - nie ma idealnej korelacji cech.

Testowanie tych własności nie zawsze jest oczywiste, a w szczególności Scikit-learn oferuje tutaj dość mało opcji, bo pochodzą one głównie z tradycyjnej statystyki.

1. Liniowość:
  - numerycznie: wysoki współczynnik dopasowania modelu $R^2$ na zbiorze treningowym, niski błąd (RMSE) na zbiorze treningowym oraz testowym
  - testem statystycznym: [Rainbow test](https://www.statsmodels.org/dev/generated/statsmodels.stats.diagnostic.linear_rainbow.html) lub [Harvey Collier test](https://www.statsmodels.org/stable/generated/statsmodels.stats.diagnostic.linear_harvey_collier.html)
  - graficznie: możliwe kiedy mamy 1/2 zmienne i da się narysować wykres zmiennej zależnej względem cech
2. Normalność błędów:
  - graficznie: robimy histogram rezyduów, powinien mieć kształt rozkładu normalnego i być wycentrowany na zerze
  - testem statystycznym: [Jarque-Bera test](https://en.wikipedia.org/wiki/Jarque%E2%80%93Bera_test), [Omnibus normality test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.normaltest.html)
3. Homoskedastyczność:
  - graficznie: robimy scatter plot rezyduów dla wartości przewidywanych od najmniejszej do największej, nie powinno być na nim żadnych widocznych wzorców czy kształtów; [przykład 1](https://towardsdatascience.com/multivariant-linear-regression-e636a4f99b40), [przykład 2](https://www.vexpower.com/brief/homoskedasticity)
  - testem statystycznym: [Breusch–Pagan test](https://en.wikipedia.org/wiki/Breusch%E2%80%93Pagan_test) lub [Goldfeld-Quandt test](https://en.wikipedia.org/wiki/Goldfeld%E2%80%93Quandt_test)
4. Niezależność błędów - nie omawiam, bo dotyczy tylko szeregów czasowych.
5. Brak współliniowości zmiennych: numerycznie, sprawdzić korelacje zmiennych, lub współczynnik uwarunkowania macierzy `X`


W ramach zadania wytrenuj model regresji liniowej dla zbioru danych Ames Housing z użyciem biblioteki Statsmodels: [OLS docs](https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.OLS.html), [OLS](https://www.statsmodels.org/dev/examples/notebooks/generated/ols.html), [Regression diagnostics](https://www.statsmodels.org/dev/examples/notebooks/generated/regression_diagnostics.html). Wytrenuj najpierw model bez regularyzacji, a następnie z regularyzacją L2 oraz L1. Nie przeprowadzaj tuningu, użyj tych wartości siły regularyzacji, które wyznaczyliśmy wcześniej.

Przetestuj założenia za pomocą testów statystycznych: Harvey Collier, Jarque-Bera, Breusch–Pagan. Współliniowość zmiennych zweryfikuj z użyciem współczynnika uwarunkowania. Zastosuj poziom istotności $\alpha=0.05$.

Czy założenia są spełnione w przypadku podstawowego modelu i/lub modeli z regularyzacją? Czy modele regularyzowane w lepszym stopniu spełniają założenia?

# 1. Przygotowanie danych

## 1.1. Czyszczenie

In [2]:
df = pd.read_csv('ames_data.csv')
df.columns = [col.replace('.', '') for col in df.columns]

In [3]:
df = df.drop(["Order", "PID"], axis="columns")
df = df.loc[~df["Neighborhood"].isin(["GrnHill", "Landmrk"]), :]
df = df.loc[df["GrLivArea"] < 4000, :]

## 1.2. Wartości brakujące

In [4]:
def replace_na(df: pd.DataFrame, col: str, value) -> None:
    df.loc[:, col] = df.loc[:, col].fillna(value)

In [5]:
default_values = {
    "Alley": "None",
    "BedroomAbvGr": 0,
    "BsmtQual": "No",
    "BsmtCond": "No",
    "BsmtExposure": "No",
    "BsmtFinType1": "No",
    "BsmtFinType2": "No",
    "BsmtFullBath": 0,
    "BsmtHalfBath": 0,
    "BsmtUnfSF": 0,
    "Condition1": "Norm",
    "Condition2": "Norm",
    "ExterCond": "TA",
    "ExterQual": "TA",
    "Fence": "No",
    "Functional": "Typ",
    "GarageType": "No",
    "GarageFinish": "No",
    "GarageQual": "No",
    "GarageCond": "No",
    "GarageArea": 0,
    "GarageCars": 0,
    "HalfBath": 0,
    "HeatingQC": "Ta",
    "KitchenAbvGr": 0,
    "KitchenQual": "TA",
    "LotFrontage": 0,
    "LotShape": "Reg",
    "MasVnrType": "None",
    "MasVnrArea": 0,
    "MiscFeature": "No",
    "MiscVal": 0,
    "OpenPorchSF": 0,
    "PavedDrive": "N",
    "PoolQC": "No",
    "PoolArea": 0,
    "SaleCondition": "Normal",
    "ScreenPorch": 0,
    "TotRmsAbvGrd": 0,
    "Utilities": "AllPub",
    "WoodDeckSF": 0,
    "CentralAir": 0,
    "EnclosedPorch": 0,
    "FireplaceQu": 0,
    "Fireplaces": 0,
    "SaleCondition": "Normal"
}

In [6]:
for column, value in default_values.items():
    replace_na(df, column, value)

In [7]:
df = df.replace({
    "MSSubClass": {
        20: "SC20",
        30: "SC30",
        40: "SC40",
        45: "SC45",
        50: "SC50",
        60: "SC60",
        70: "SC70",
        75: "SC75",
        80: "SC80",
        85: "SC85",
        90: "SC90",
        120: "SC120",
        150: "SC150",
        160: "SC160",
        180: "SC180",
        190: "SC190",
    },
    "MoSold": {
        1: "Jan",
        2: "Feb",
        3: "Mar",
        4: "Apr",
        5: "May",
        6: "Jun",
        7: "Jul",
        8: "Aug",
        9: "Sep",
        10: "Oct",
        11: "Nov",
        12: "Dec",
    },
    "Alley": {"None": 0, "Grvl": 1, "Pave": 2},
    "BsmtCond": {"No": 0, "Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5},
    "BsmtExposure": {"No": 0, "Mn": 1, "Av": 2, "Gd": 3},
    "BsmtFinType1": {
        "No": 0,
        "Unf": 1,
        "LwQ": 2,
        "Rec": 3,
        "BLQ": 4,
        "ALQ": 5,
        "GLQ": 6,
    },
    "BsmtFinType2": {
        "No": 0,
        "Unf": 1,
        "LwQ": 2,
        "Rec": 3,
        "BLQ": 4,
        "ALQ": 5,
        "GLQ": 6,
    },
    "BsmtQual": {"No": 0, "Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5},
    "ExterCond": {"Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5},
    "ExterQual": {"Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5},
    "FireplaceQu": {"No": 0, "Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5},
    "Functional": {
        "Sal": 1,
        "Sev": 2,
        "Maj2": 3,
        "Maj1": 4,
        "Mod": 5,
        "Min2": 6,
        "Min1": 7,
        "Typ": 8,
    },
    "GarageCond": {"No": 0, "Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5},
    "GarageQual": {"No": 0, "Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5},
    "HeatingQC": {"Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5},
    "KitchenQual": {"Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5},
    "LandSlope": {"Sev": 1, "Mod": 2, "Gtl": 3},
    "LotShape": {"IR3": 1, "IR2": 2, "IR1": 3, "Reg": 4},
    "PavedDrive": {"N": 0, "P": 1, "Y": 2},
    "PoolQC": {"No": 0, "Fa": 1, "TA": 2, "Gd": 3, "Ex": 4},
    "Street": {"Grvl": 0, "Pave": 1},
    "Utilities": {"ELO": 1, "NoSeWa": 2, "NoSewr": 3, "AllPub": 4},
})

## 1.3. Transformacje

In [8]:
y = df.pop("SalePrice")

In [9]:
numerical_features = df.select_dtypes(exclude='object').columns
categorical_features = df.select_dtypes(include='object').columns

In [10]:
one_hot_encoder = OneHotEncoder(
    drop="first", 
    sparse_output=False, 
    handle_unknown="ignore"
)
median_imputer = SimpleImputer(strategy="median")
min_max_scaler = MinMaxScaler()

categorical_pipeline = Pipeline([('encoding', one_hot_encoder)])

numerical_pipeline = Pipeline([
    ('imputation', median_imputer), 
    ('scaling', min_max_scaler)
])

column_transformer = ColumnTransformer([
    ('numerical', numerical_pipeline, numerical_features),
    ('categorical', categorical_pipeline, categorical_features)
])

column_transformer.fit(df, y)

ColumnTransformer(transformers=[('numerical',
                                 Pipeline(steps=[('imputation',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaling', MinMaxScaler())]),
                                 Index(['LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'Utilities',
       'LandSlope', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond',
       'BsmtExposure',...
                                                                sparse_output=False))]),
                                 Index(['MSSubClass', 'MSZoning', 'LandContour', 'LotConfig', 'Neighborhood',
       'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation',
       'Heating', 'CentralAir', 'Electrical', 'GarageType', 'GarageFinish',
       'Fence', 'MiscFeature', 'MoSold', 'SaleType', 'SaleCondition'],
      dtype='object'))])

In [11]:
df = column_transformer.transform(df)

# 2. Badanie własności

In [77]:
indent = 4 * ' '

def display_test(model, title, stats_names, test, *args, **kwargs):
    score = test(model, *args, **kwargs)

    text = f'{title} test:\n'

    for name, stat in zip(stats_names, score):
        text += f'{indent}{name}: {stat}\n'

    print(text)

In [37]:
model = sm.OLS(
    y,
    df,
    hasconst=False
).fit()

## 2.1. Liniowa zależność

### 2.1.1. Rainbow

In [73]:
stats = ['score', 'p-value']

display_test(model, 'Rainbow', stats, linear_rainbow)

Rainbow test:
    score: 1.2228654609712235
    p-value: 0.00012288105014709246



Bardzo mała p-wartość ($p << \alpha$) w teście Rainbow sugeruje, że hipoteza zerowa jest fałszywa i rzeczywista zależność między danymi nie jest liniowa

### 2.1.2. Harvey-Collier

In [72]:
stats = ['score', 'p-value']

try:
    display_test(model, 'Harvey Collier', stats, linear_harvey_collier, skip=df.shape[0])
except Exception as e:
    print(e)

"The initial regressor matrix, x[:skip], issingular. You must use a value of
skip large enough to ensure that the first OLS estimator is well-defined.



## 2.2. Rozkład błędu

In [71]:
stats = ['score', 'p-value', 'skew', 'kurtosis']

try:
    display_test(model.resid, 'Jarque-Bera', stats, jarque_bera)
except Exception as e:
    print(e)

Jarque-Bera test:
    score: 6176.755455935331
    p-value: 0.0
    skew: 0.5004648832785793
    kurtosis: 10.052038284303693



Estymaty skośności i kurtozy znacząco odbiegają od wartości dla rozkładu normalnego. Ponadto $p << \alpha$

## 2.3. Homoskedastyczność

In [70]:
stats = ['LM score', 'LM p-value', 'F score', 'F p-value']

try:
    display_test(model.resid, 'Breusch-Pagan', stats, het_breuschpagan, exog_het=model.model.exog)
except Exception as e:
    print(e)

The Breusch-Pagan test requires exog to have at least two columns where one is a constant.


## 2.4. Współliniowość zmiennych

In [53]:
model.condition_number

7.380609343770715e+16

Wysoka wartość współczynnika uwarunkowania świadczy o istnieniu par silnie skorelowanych predyktorów